# Lab 5 - Parcel Feature Extraction

Next, we will illustrate the construction of features related to our main task: finding the relationship between property development and water quality over time.  In a previous lab, you identified lakes for which we have complete information for the years from 2004 to 2015.  In this lab, we will

[Original Data and variable information](https://gisdata.mn.gov/organization/us-mn-state-metrogis?q=Metro+Regional+Parcel+Dataset&sort=score+desc%2C+metadata_modified+desc)

## Problem 1 - Feature construction

**Overview.** Remember that our target output file will have one row per year-lake combination.  To attach property information, we will need to group and aggregate the parcel data to create features for each lake-year combination.  When grouping the data, be sure to maintain the variables needed to join to the water quality data, namely the lake ID and year.  Since we are looking at tracking property development/change over time, we will want to generate features tracking

* Number of properties close to each lake,
* Summaries of the value of properties close to each lake,
* Aggregations on the size and type of the properties, and
* Other features that might impact water quality.
    
#### Task 1. Understanding parcel variables

Before we can construct features, we need to make sure we understand the parcel data.  The metro parcel data is provided by the State of Minnesota and the meta data can be found online.  For example, searching for *metro parcel 2014* lead to [this site](https://geo.btaa.org/catalog/304cf3d8-a53b-4ea9-b02a-f550bd68e320).  Clicking on the *Meta data* button in the top left, brought up more information.  Clicking *Download* opened in this meta data [in a separate page](https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_parcels_2014/metadata/metadata.html)

Look through the **Section 4: Attributes** and identify variables that might impact the water quality of near-by lakes.

> <font color="orange"> Your thoughts here </font>

The Minnesota Metro Parcel dataset provides variables that help link property development to lake water quality. Key attributes include parcel IDs and location data, land use codes, parcel size, building type, and year built, which show the type and intensity of development. Assessed property values and tax classifications reflect economic activity, while environmental indicators like impervious surface coverage, agricultural use, and vacant land status highlight potential impacts on runoff. Ownership details, such as public versus private land, also matter since management practices differ. Together, these variables allow you to track how property changes may affect water quality over time.

### Task 2. Feature Brainstorming

Our objective is to build a feature table with one row per lake-year, using grouped summary statistics. Here are effective strategies for feature construction:

1. **Numerical summaries:** Calculate group-level statistics (mean, median, standard deviation, IQR, etc.) for numeric variables.
2. **Categorical summaries:** For text data, consider:
   - **Success rates:** Compute proportions for binary variables (e.g., percent of homes with basements).
   - **Label cleaning:** Review and standardize unique labels to remove duplicates or inconsistencies.
   - **Broader categories:** Recode variables with many rare categories into a smaller, more meaningful set.
   - **Indicator columns:** Create indicator variables and aggregate them to show presence/absence or proportions (e.g., count of each property use type).

Review the variables you identified earlier and outline a feature construction strategy for each.

> <font color="orange"> Your thoughts here </font>

To build the lake‑year feature table, numeric variables like parcel size, property value, year built, and impervious area should be summarized with means, medians, totals, and variability. Categorical variables such as land use, building type, tax class, and ownership can be cleaned, grouped into broader categories, and aggregated into proportions or indicators. Binary attributes like vacant land, agricultural use, or homestead status are best expressed as counts or percentages. This approach ensures each lake‑year record captures both the scale and type of development affecting water quality.

### Task 4. Initial querying with filter and select

First, you should build a query that filters the parcel data to
1. only include parcels within 1600 feet of the lakes we are studying, and
2. only for the lakes with complete information.  

You should also select only the columns you will need for feature construction and joining to the water quality data.

In [ ]:
import polars as pl
import polars.selectors as cs
from glob import glob
import re

In [24]:
import polars as pl
import polars.selectors as cs
from glob import glob
import re

(mudac_17 := (
    pl.scan_csv(
        'data/MinneMUDAC_raw_files/Final_Data_from_MUDAC17.csv',
        has_header=True,
        separator=',',
        infer_schema_length=10000,
        schema_overrides={'PROC_Num_OBSTETRICAL_PROCEDURES': pl.Float64}
    )
    .limit(100)
    .collect()
))

mudac_17

PATID,Gender,YOB,Age,STATE,Num_Claims,TOTAL COST MEDICAL (unadj),Num_POS_Claims,TOTAL COST MEDICAL,Total Cost Training,STD_COST_ACUTE-CARE_INPATIENT_FACILITY,STD_COST_ACUTE-CARE_OUTPATIENT_FACILITY,STD_COST_HOSPICE,STD_COST_INDEPENDENT_LABORATORY,STD_COST_INPATIENT_REHAB,STD_COST_LONG-TERM_CARE_FACILITY,STD_COST_Missing,STD_COST_OFFICE/CLINIC,STD_COST_OTHER_OUTPATIENT,STD_COST_OTHER_UNLISTED_FACILITY,Num_ACUTE-CARE_INPATIENT_FACILITY,Num_ACUTE-CARE_OUTPATIENT_FACILITY,Num_HOSPICE,Num_INDEPENDENT_LABORATORY,Num_INPATIENT_REHAB,Num_LONG-TERM_CARE_FACILITY,Num_Missing,Num_OFFICE/CLINIC,Num_OTHER_OUTPATIENT,Num_OTHER_UNLISTED_FACILITY,DIAG_Num_ALCOHOL/DRUG_USE_&_ALCOHOL/DRUG_INDUCED_ORGANIC_M,DIAG_Num_BURNS,DIAG_Num_DISEASES_&_DISORDERS_OF_BLOOD_BLOOD_FORMING_ORGA,DIAG_Num_DISEASES_&_DISORDERS_OF_THE_CIRCULATORY_SYSTEM,DIAG_Num_DISEASES_&_DISORDERS_OF_THE_DIGESTIVE_SYSTEM,DIAG_Num_DISEASES_&_DISORDERS_OF_THE_EAR_NOSE_MOUTH_&_TH,DIAG_Num_DISEASES_&_DISORDERS_OF_THE_EYE,…,"No, RESPIRATORY AND CNS STIMULANTS","No, SECOND GENERATION ANTIHISTAMINES","No, SEL. SEROTONIN & NOREPI REUPTAKE INHIBTR","No, SELECTIVE BETA-2-ADRENERGIC AGONISTS","No, SELECTIVE SEROTONIN AGONISTS","No, SELECTIVE-SEROTONIN REUPTAKE INHIBITORS","No, SEROTONIN MODULATORS","No, SKIN AND MUCOUS MEMBRANE AGENTS, MISC.","No, SULFONAMIDES (SYSTEMIC","No, TETRACYCLINES","No, THIAZIDE DIURETICS","No, THIRD GENERATION CEPHALOSPORINS","No, THYROID AGENTS","No, TRICYCLICS, OTHER NOREPI-RU INHIBITORS","No, URINARY ANTI-INFECTIVES","No, VITAMIN B COMPLEX","No, VITAMIN D","Yes, BIGUANIDES","Yes, DIPEPTIDYL PEPTIDASE-4(DPP-4 INHIBITORS","Yes, INCRETIN MIMETICS","Yes, INSULINS","Yes, Other","Yes, SODIUM-GLUC COTRANSPORT 2 (SGLT2 INHIB","Yes, SULFONYLUREAS","Yes, THIAZOLIDINEDIONES",Num_LABS,LAB_A,LAB_H,LAB_L,LAB_N,LAB_None,LAB_UN,Ranked Total Cost Training,In Top 6000 Training,Total Cost Target,Ranked Total Cost Target,In Top 6000 Target
i64,str,i64,i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,f64,i64,i64
33004375180,"""F""",1990,27,"""OH""",9,720.36,3.0,240.12,588.853333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,240.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.333,0.333,0.0,0.667,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3725,0,0.0,2,0
33004540916,"""F""",1985,32,"""OH""",64,21684.84,21.33,7228.28,7307.886667,0.0,6164.75,0.0,104.79,0.0,0.0,0.0,958.74,0.0,0.0,0.0,4.667,0.0,4.667,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.667,0.0,…,0.0,0.0,0.0,0.0,1.333,0.0,0.0,0.0,0.0,0.333,0.0,0.0,0.0,0.667,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28953,0,3464.08,22950,0
33004570387,"""F""",1977,40,"""FL""",58,10223.52,19.33,3407.84,3615.473333,2565.87,306.63,0.0,103.63,0.0,0.0,0.0,431.71,0.0,0.0,8.667,2.0,0.0,4.333,0.0,0.0,0.0,4.333,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.333,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333,4.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.333,0.667,1.0,0.667,0.0,43.0,0.0,21098,0,1014.95,11275,0
33004940807,"""F""",1984,33,"""MO""",28,1522.5,9.33,507.5,534.646667,0.0,0.0,0.0,158.92,0.0,0.0,0.0,348.58,0.0,0.0,0.0,0.0,0.0,5.667,0.0,0.0,0.0,3.667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.667,1.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.333,0.333,0.667,0.333,0.0,35.0,0.0,3274,0,1684.1,15856,0
33005012652,"""F""",1984,33,"""RI""",48,7868.8,16.0,2622.93,2986.083333,0.0,635.62,0.0,24.75,0.0,0.0,0.0,1875.76,0.0,86.81,0.0,2.333,0.0,0.333,0.0,0.0,0.0,13.0,0.0,0.333,0.0,0.0,1.0,0.333,1.0,4.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18804,0,788.56,9280,0
…,…,…,…,…,…,…,…,…,

## Problem 2.  Numerical Summaries

Two important categories of property data involve the size (e.g., finished square footage) and value (e.g., accessed value and/or taxes paid).

**Tasks.**

1. Identify 2-3 variables for each of these categories.
2. Write a query that computes the summary statistics for each of these variables for each lake-year.  
3. Write this summary table out to a CSV file named `parcel_numerical_summaries.csv`.  Again, you should partition by lake ID and year.

In [ ]:
# Your code here

## Problem 3.  Simple categorical summaries.

In this part, you will create summary statistics for some of the simpler categorical variables.

**Binary variables.** There are two examples of binary variables, listed below.  You will need to compute the percent of `Yes` for each.

* GARAGE: Garage Y/N
* BASEMENT: Basement Y/N

**Other categorical variables.** There are a number of other categorical variables.  You need to select one of these variables, inspect/clean your variable as needed, create indicator variables for each resulting label, and compute summary statistics for each label.

* HOMESTEAD: Homestead Status
* TAX_EXEMPT: Tax Exempt Status
* DWELL_TYPE: Dwelling Type
* HOME_STYLE: Home Style
* HEATING: Heating type
* COOLING: Cooling type

**Tasks.**
Create a query that

1. Select one binary and two other categorical variables for feature construction,
2. Reads in the parcel data and selects the relevant columns (be sure to keep the lake ID and year),
3. Inspect unique labels and recode/clean as needed,
4. Create a literal column of ones, and
5. Pivot to get the counts of each label per lake-year (do this once per category).

Write this summary table out to a csv file named `parcel_categorical_summaries.csv`.  Again, you should partition by lake ID and year.

In [ ]:
# Your code here

## Problem 4.  Join all the summaries.

Finally, you need to join all the summaries created above, along with the water quality summaries created in a previous lab, into one overall summary file.  Write the resulting table to a CSV file named `water_quality_and_parcel_summaries_2004_to_2015.csv`.

In [ ]:
# Your code here.

## Problem 5.  Put it all together

It is often useful to package all of the data constructions steps together in one convenient place.  Your last task is to

1. Gather all of your data construction code below.
    * You don't need to include exploratory code, e.g., exploring join mismatches; only the code necessary to combine, clean, and write your data.
2. Clean/refactor the code.
3. Be sure to display all important intermediate results.

In [ ]:
# Your code here.